# FullHouse + Quads sanity checks

Minimal correctness + integration checks for the new claim kinds.

In [ ]:
import os, sys
from pathlib import Path

# Add repo root to sys.path

def find_repo_root(start_dir: str) -> str:
    cur = Path(start_dir).resolve()
    for _ in range(6):
        if (cur / "liars_poker").is_dir() or (cur / "pyproject.toml").exists():
            return str(cur)
        if cur.parent == cur:
            break
        cur = cur.parent
    return str(Path(start_dir).resolve())

NB_DIR = Path.cwd()
REPO_ROOT = Path(find_repo_root(NB_DIR))
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

In [ ]:
from liars_poker.core import GameSpec
from liars_poker.env import Rules, resolve_call_winner
from liars_poker.infoset import CALL


## Claim ordering + FullHouse index list

In [ ]:
spec = GameSpec(
    ranks=3,
    suits=4,
    hand_size=2,
    claim_kinds=("RankHigh", "Pair", "TwoPair", "Trips", "FullHouse", "Quads"),
    suit_symmetry=False,
)
rules = Rules(spec)

first_idx = {k: next(i for i, (kind, _) in enumerate(rules.claims) if kind == k) for k in {
    "RankHigh", "Pair", "TwoPair", "Trips", "FullHouse", "Quads"
}}

assert first_idx["RankHigh"] < first_idx["Pair"] < first_idx["TwoPair"] < first_idx["Trips"] < first_idx["FullHouse"] < first_idx["Quads"]

fh = rules.full_house_ranks
assert len(fh) == spec.ranks * (spec.ranks - 1)
assert fh[0] == (1, 2)
assert fh[-1] == (spec.ranks, spec.ranks - 1)

print("ordering ok", first_idx)
print("fullhouse list ok", fh[0], fh[-1])


## resolve_call_winner sanity

In [ ]:
spec_full = GameSpec(
    ranks=4,
    suits=4,
    hand_size=3,
    claim_kinds=("RankHigh", "Pair", "TwoPair", "Trips", "FullHouse", "Quads"),
    suit_symmetry=False,
)

rules_full = Rules(spec_full)

def card(rank, suit):
    return (rank - 1) * spec_full.suits + suit

# FullHouse true: trip=4, pair=3
p1_hand = (card(4, 0), card(4, 1), card(4, 2))
p2_hand = (card(3, 0), card(3, 1), card(1, 0))
fh_action = rules_full.parse_action("FullHouse:4,3")
winner = resolve_call_winner(spec_full, (fh_action, CALL), p1_hand, p2_hand)
assert winner == "P1"

# FullHouse false: trip=4, pair=2 (pair missing)
fh_action_false = rules_full.parse_action("FullHouse:4,2")
winner = resolve_call_winner(spec_full, (fh_action_false, CALL), p1_hand, p2_hand)
assert winner == "P2"

# Quads true: rank=2
p1_hand = (card(2, 0), card(2, 1), card(1, 1))
p2_hand = (card(2, 2), card(2, 3), card(3, 0))
q_action = rules_full.parse_action("Quads:2")
winner = resolve_call_winner(spec_full, (q_action, CALL), p1_hand, p2_hand)
assert winner == "P1"

print("resolve_call_winner ok")


## Dense FSP smoke test (episodes_test != 0)

In [ ]:
from liars_poker.training.dense_fsp import dense_fsp_loop

spec_small = GameSpec(
    ranks=2,
    suits=4,
    hand_size=2,
    claim_kinds=("RankHigh", "Pair", "TwoPair", "Trips", "FullHouse", "Quads"),
    suit_symmetry=False,
)

policy, info = dense_fsp_loop(
    spec=spec_small,
    episodes=2,
    episodes_test=200,
    debug=False,
)
print("dense_fsp_loop ok", list(info.keys()))


## CFR / CFR+ smoke tests

In [ ]:
from liars_poker.training.cfr_dense import cfr_dense_loop
from liars_poker.training.cfr_plus_dense import cfr_plus_loop

policy_cfr, logs_cfr, _ = cfr_dense_loop(
    spec=spec_small,
    iterations=3,
    eval_every=0,
    debug=False,
)
print("cfr_dense_loop ok", len(logs_cfr.get("exploitability_series", [])))

policy_cfrp, logs_cfrp, _ = cfr_plus_loop(
    spec=spec_small,
    iterations=3,
    eval_every=0,
    debug=False,
)
print("cfr_plus_loop ok", len(logs_cfrp.get("exploitability_series", [])))
